In [1]:
# imports
import os
import imp
import numpy as np
import pandas as pd
import theano
import lasagne
import loading
from training import *
from network import *
from architectures import *
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import bayes_mvs as bmvs

%matplotlib inline

# aliases
L = lasagne.layers
nl = lasagne.nonlinearities
T = theano.tensor

In [9]:
# data loading
headdir = os.path.expanduser('~/Google Drive/Bas Zahy Gianni - Games')
paramsdir = os.path.join(headdir, 'Analysis/0_hvh/Params/nnets/temp')
datadir = os.path.join(headdir, 'Data/model input')
# datafile = os.path.join(headdir, 'Data/0_hvh/Clean/_summaries/model_input_with_groups.csv')
resultsdir = os.path.join(headdir, 'Analysis/0_hvh/Loglik/nnets')
# data = loading.default_loader(datafile)

datafilenames = ['0 (with groups)', '1 (with computer)', '2 (with computer)', '3 (with computer)', '4']
datafilenames = [os.path.join(datadir, fname + '.csv') for fname in datafilenames]
colnames = ['subject', 'color', 'bp', 'wp', 'zet', 'rt']

e0 = pd.read_csv(datafilenames[0], names=colnames+['splitno'])
e1 = pd.read_csv(datafilenames[1], names=colnames)
e2 = pd.read_csv(datafilenames[2], names=colnames)
e3 = pd.read_csv(datafilenames[3], names=colnames+['task', 'taskorder', 'session'])
e4 = pd.read_csv(datafilenames[4], names=colnames+['timecondition'])

Es = [e1, e2, e3, e4]
for i, e in enumerate(Es[1:]):
    e['subject'] = e['subject'] + Es[i-1].loc[Es[i-1]['subject']<1000, 'subject'].max()

A = pd.concat([e[colnames] for e in [e1, e2, e3, e4]])

groups = np.arange(len(A))%5 + 1
np.random.seed(100001)
np.random.shuffle(groups)
A['group'] = groups

A.to_csv(os.path.join(datadir, '1-4.csv'), encoding='ASCII', header=False, index=False)
A.loc[A['subject']<1000, :].to_csv(
    os.path.join(datadir, '1-4 (no computer).csv'), 
    encoding='ASCII', header=False, index=False
)

In [10]:
data = loading.default_loader(os.path.join(datadir, '1-4 (no computer).csv'))

# Prototyping

### Baseline CNN performance on Experiments 1-4

In [ ]:
def prototype(input_var=None):
    input_shape = (None, 2, 4, 9)
    FixLayer = make_FixLayer(input_var)

    input_layer = L.InputLayer(shape=input_shape, input_var=input_var)
    network = L.Conv2DLayer(
        input_layer, num_filters=32, filter_size=(4,4), pad='full',
        nonlinearity=nl.identity
    )

    network = L.ParametricRectifierLayer(network, shared_axes='auto') # default: auto
    network = L.FeaturePoolLayer(network, pool_function=T.sum, pool_size=2) # default: T.sum, 2
    network = L.DropoutLayer(network, p=.75) # default: .75
    network = L.DenseLayer(
        network, num_units=36,
        nonlinearity=nl.very_leaky_rectify, W=lasagne.init.HeUniform(gain='relu')
    )
    network = FixLayer(network)
    network = L.NonlinearityLayer(network, nonlinearity=nl.softmax)
    network = FixLayer(network)
    network = ReNormLayer(network)

    return network

trainer = DefaultTrainer(stopthresh=100) # default: 125
net_list = trainer.train_all(architecture=prototype, data=data, seed=985227)

for i, n in enumerate(net_list):
    n.save_params(os.path.join(paramsdir, '{} split agg fit exp 1-4'.format(i)))

(12362, 2, 4, 9)
Epoch 0 took 16.673s
	training loss:			2.6981
	validation loss:		2.5322
	validation accuracy:		22.75%
	total time elapsed:		16.945s
Epoch 50 took 15.195s
	training loss:			2.1220
	validation loss:		1.9659
	validation accuracy:		40.54%
	total time elapsed:		795.825s
Epoch 100 took 15.196s
	training loss:			2.1086
	validation loss:		1.9451
	validation accuracy:		41.02%
	total time elapsed:		1570.664s
Epoch 150 took 15.346s
	training loss:			2.0972
	validation loss:		1.9463
	validation accuracy:		41.43%
	total time elapsed:		2364.337s
Epoch 200 took 15.640s
	training loss:			2.0945
	validation loss:		1.9386
	validation accuracy:		41.49%
	total time elapsed:		3147.492s
Abandon ship!
TEST PERFORMANCE
	Stopped in epoch:		231
	Test loss:			1.9396
	Test accuracy:			41.59%
(12396, 2, 4, 9)
Epoch 0 took 16.167s
	training loss:			2.6881
	validation loss:		2.5109
	validation accuracy:		24.21%
	total time elapsed:		16.419s
